<a href="https://colab.research.google.com/github/Yutongzhang20080108/Chinese-text-generation/blob/main/NAIVE_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install langsmith
!pip install langchain_openai
!pip install langchain_chroma
!pip install langchain_community
!pip install langchainhub

In [6]:
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(base_url="https://api.chatanywhere.tech/v1",model="gpt-3.5-turbo-0125")

··········


In [68]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
with open("passage.txt", "r", encoding="utf-8") as f:
  passage = f.read()
print(len(passage))
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=5,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    # Existing args
)
texts = text_splitter.create_documents([passage])
print(texts[0])
print(texts[1])

6239
page_content='淑德延公胄，宜家接帝姻。桂宫男掌仆，兰殿女升嫔。恩泽昭前命'
page_content='，盈虚变此辰。万年今已矣，彤管列何人。虫响乱啾啾'


In [69]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=texts, embedding=OpenAIEmbeddings(base_url="https://api.chatanywhere.tech/v1"))

In [40]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("请你围绕流水写一首诗歌")

print(retrieved_docs)

[Document(page_content='，春风送客使人悲。怨别自惊千里外，论交却忆十年时。云开汶水孤帆远，路绕梁山匹马迟。此地从来可乘兴，留君不住益凄其。高馆张灯酒复清，夜钟残月雁归声。只言啼鸟堪求侣，无那春风欲送行。黄河曲里沙为岸，白马津边柳向城。莫怨他乡暂离别，知君到处有逢迎。嗟君此别意何如，驻马衔桮问谪居。巫峡啼猿数行泪，衡阳归雁几封书。青枫江上秋天远，白帝城边古木疎。圣代即今多雨露，暂时分手莫踌躇。同官载酒出郊圻，晴日东驰雁北飞。隔岸春云邀翰墨，傍簷垂柳报芳菲。池边转觉虚无尽，台上偏宜酩酊归。州县徒劳那可度，后时连骑莫相违。北楼西望满晴空，积水连山胜画中。湍上急流声若箭，城头残月势如弓。垂竿已羡磻谿老，体道犹思塞上翁。为问边庭更何事，至今羌笛怨无穷。传君昨夜怅然悲，独坐新斋木落时。逸气旧来凌燕雀，高才何得混妍媸。迹留黄绶人多叹，心在青云世莫知。不是鬼神无正直，从来州县有瑕疵。节物惊心两鬓华，东篱空绕未开花。百年将半仕三已，五亩就荒天一涯。岂有白衣来剥啄，一从乌帽自攲斜。真成独坐空搔首，门柳萧萧噪暮鸦。德以精灵降，时膺梦寐求。苍生谢安石，天子富平侯。尊俎资高论，岩廊挹大猷。相门连户牖，卿族嗣弓裘。豁达云开霁，清明月映秋。能为吉甫颂，善用子房筹。阶砌思攀陟，门阑尚阻修。高山不易仰，大匠本难投。迹与松乔合，心缘启沃留。公才山吏部，书癖杜荆州。幸沐千年圣，何辞一尉休。折腰知宠辱，廻首见沈浮。天地庄生马，江湖范蠡舟。逍遥堪自乐，浩荡信无忧。去此从黄绶，归欤任白头。风尘与霄汉，瞻望日悠悠。风俗登淳古，君臣挹大庭。深沈谋九德，密勿契千龄。独立调元气，清心豁窅冥。本枝连帝系，长策冠生灵。傅说明殷道，萧何律汉刑。钧衡持国柄，柱石总朝经。隐轸江山藻，氛氲鼎鼐铭。兴中皆白雪，身外即丹青。江海呼穷鸟，诗书问聚萤。吹嘘成羽翼，提握动芳馨。倚伏悲还笑，栖迟醉复醒。恩荣初就列，含育忝宵形。有窃丘山惠，无时枕席宁。壮心瞻落景，生事感浮萍。莫以才难用，终期善易听。未为门下客，徒谢少微星。遥传副丞相，昨日破西蕃。作气群山动，扬军大斾翻。奇兵邀转战，连孥绝归奔。泉喷诸戎血，风驱死虏魂。头飞攒万戟，面缚聚辕门。鬼哭黄埃暮，天愁白日昏。石城与岩险，铁骑皆云屯。长策一言决，高踪百代存。威棱慑沙漠，忠义感乾坤。老将黯无色，儒生安敢论。解围凭庙算，止杀报君恩。唯有关河渺，苍茫空树墩。云纪轩皇代，星

In [73]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate

system_template = "你是一个诗歌生成助手。请你使用收集到的内容来回答这些问题。如果你不知道答案，你只需要说你不知道。"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template+"\n"),("user","\nQuestion: "+"{question}"+"\nContext: "+"{context}"+"\nAnswer:")]
)

example_messages = prompt_template.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[SystemMessage(content='你是一个诗歌生成助手。请你使用收集到的内容来回答这些问题。如果你不知道答案，你只需要说你不知道。\n'),
 HumanMessage(content='\nQuestion: filler question\nContext: filler context\nAnswer:')]

In [83]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("请你围绕天空写一首七言律诗"):
    print(chunk, end="", flush=True)

青天如水碧悠悠，白云飘散舞轻柔。徜徉其中心自逸，凝望高处思无忧。

In [92]:
grade_llm = ChatOpenAI(base_url="https://api.chatanywhere.tech/v1", model="gpt-4")
system_template = "你是一个诗歌评估助手。请你根据以下评价要求对于给出的诗歌进行评价。请你只需要给出最后数字而不需要解释。评价标准：（评分等级为1到5）：①格式。生成的唐诗是否遵循写作规则,尤其注意对仗、句数、字数、平仄；②创新性。生成的唐诗是否抄袭现成的诗句，是否具有创新性；③相关性。生成的唐诗与给定主题是否相关；④美学。生成的唐诗是否符合中国诗词的审美，包括寓情于物等；⑤整体。从整体情况评价生成的唐诗。"
prompt = ChatPromptTemplate.from_messages(
    [("system", system_template+"\n"),("user","\nQuestion: "+"{question}"+"\nAnswer")]
)
grade_chain = prompt | grade_llm | StrOutputParser()

In [93]:
theme = ["流水","天空","树木","爱情","自由"]
score = []
passage = []
for each in theme:
  response = rag_chain.invoke("请你围绕天空写一首七言律诗")
  passage.append(response)
  scores = grade_chain.invoke(response)
  score.append(scores)
print(score)
print(passage)

['①格式：4\n②创新性：3\n③相关性：4\n④美学：4\n⑤整体：4', '①格式：4\n②创新性：4\n③相关性：4\n④美学：4\n⑤整体：4', '①格式：4\n②创新性：4\n③相关性：5\n④美学：5\n⑤整体：4', '①格式：4\n②创新性：4\n③相关性：4\n④美学：4\n⑤整体：4', '①格式：4\n②创新性：4\n③相关性：4\n④美学：3\n⑤整体：4']
['天空深邃浩渺宽，星云交缠亦无关。风吹云卷天无边，晴空万里白云间。', '天空苍茫云涌动，朔气连天黑不通。清心望云端，羽衣相识长无极。', '天空无边浩瀚宽，云卷云舒似绸缎。碧空连天青气浓，夜幕降临星斗亮。', '天空浩瀚云雾深，征人相对思归心。黑暗中云气浮连，清心回望云端起。羽盖霓裳相识处，传情写念长无极。', '天空浩瀚云深黑，思归征人望故乡。云和朔气连天暗，清心回望云之光。羽盖霓裳传情意，长无极写念长。']
